### Malgré les apparences, c'est bien le dataset MELD qui a été utilisé

In [10]:
#!pip install rich

In [2]:
#Test avec un seul fichier audio
import librosa
import numpy as np
import joblib
import os

# Mettre le chemin du modèle ici
model = joblib.load("kaggle_save/GaussianNB.joblib")

# Ordre important
sentiment_labels = ['negative', 'neutral', 'positive']

def extract_mfcc(filepath, n_mfcc=13):
    y, sr = librosa.load(filepath, sr=None)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    return np.mean(mfcc.T, axis=0)

audio_file = "sdataset/TEST/4.wav"

features = extract_mfcc(audio_file).reshape(1, -1)
proba = model.predict_proba(features)[0]
pred_sentiment = sentiment_labels[np.argmax(proba)]

print(f"\n🎧 Fichier : {os.path.basename(audio_file)}")
print(f"🔮 Sentiment prédit : {pred_sentiment}")
print("📊 Probabilités par sentiment :")
for label, p in zip(sentiment_labels, proba):
    print(f"  {label}: {p:.4f}")



🎧 Fichier : 4.wav
🔮 Sentiment prédit : neutral
📊 Probabilités par sentiment :
  negative: 0.3273
  neutral: 0.5437
  positive: 0.1290


In [13]:
# Test avec un dossier de fichiers audio
import librosa
import numpy as np
import joblib
import os
from rich.live import Live
from rich.table import Table
from rich.console import Console

# Chargement du modèle
model = joblib.load("kaggle_save/GaussianNB.joblib")
sentiment_labels = ['negative', 'neutral', 'positive']

def extract_mfcc(filepath, n_mfcc=13):
    y, sr = librosa.load(filepath, sr=None)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    return np.mean(mfcc.T, axis=0)

test_folder = "dataset_clean/test/audios"
file_list = sorted([f for f in os.listdir(test_folder) if f.lower().endswith(".wav")])

table = Table(show_header=True, header_style="bold magenta")
table.add_column("Fichier", justify="left")
table.add_column("Prédit", justify="center")
table.add_column("Negative", justify="right")
table.add_column("Neutral", justify="right")
table.add_column("Positive", justify="right")

with Live(table, refresh_per_second=4) as live:
    for filename in file_list:
        path = os.path.join(test_folder, filename)
        features = extract_mfcc(path).reshape(1, -1)
        proba = model.predict_proba(features)[0]
        pred = sentiment_labels[np.argmax(proba)]

        table.add_row(
            filename,
            pred,
            f"{proba[0]:.4f}",
            f"{proba[1]:.4f}",
            f"{proba[2]:.4f}"
        )
        live.update(table)


Output()

KeyboardInterrupt: 

## En temps réel

In [3]:
import speech_recognition as sr
import time
import numpy as np
import joblib
import librosa
from rich.live import Live
from rich.table import Table
from rich.panel import Panel
from rich.console import Group, Console

model = joblib.load("kaggle_save/GaussianNB.joblib")
sentiment_labels = ['negative', 'neutral', 'positive']

last_transcription = ""
stop = False  # flag pour arrêter la boucle

def extract_mfcc_from_audio_data(audio_data, sr=16000, n_mfcc=13):
    mfcc = librosa.feature.mfcc(y=audio_data, sr=sr, n_mfcc=n_mfcc)
    return np.mean(mfcc.T, axis=0)

def callback(recognizer, audio):
    global last_transcription
    try:
        text = recognizer.recognize_google(audio, language="fr-FR")
        last_transcription = text

        audio_data_int16 = np.frombuffer(audio.get_raw_data(), dtype=np.int16)
        audio_data = audio_data_int16.astype(np.float32) / 32768.0

        features = extract_mfcc_from_audio_data(audio_data).reshape(1, -1)
        proba = model.predict_proba(features)[0]
        pred_sentiment = sentiment_labels[np.argmax(proba)]

        timestamp = time.strftime("%H:%M:%S")

        table.add_row(timestamp, pred_sentiment, f"{proba[0]:.4f}", f"{proba[1]:.4f}", f"{proba[2]:.4f}")

    except sr.UnknownValueError:
        last_transcription = "[Audio incompris]"
    except sr.RequestError as e:
        last_transcription = f"[Erreur service reconnaissance vocale: {e}]"
    except Exception as e:
        last_transcription = f"[Erreur: {e}]"

def main():
    global table, stop

    r = sr.Recognizer()
    m = sr.Microphone()

    table = Table(show_header=True, header_style="bold magenta")
    table.add_column("Timestamp", justify="center")
    table.add_column("Sentiment prédit", justify="center")
    table.add_column("Negative", justify="right")
    table.add_column("Neutral", justify="right")
    table.add_column("Positive", justify="right")

    with m as source:
        print("Calibrage du bruit ambiant...")
        r.adjust_for_ambient_noise(source, duration=2)

    print("Démarrage reconnaissance vocale et détection sentiment...")

    stop_listening = r.listen_in_background(m, callback)

    console = Console()

    try:
        with Live(console=console, refresh_per_second=4) as live:
            while not stop:
                panel = Panel(last_transcription or "[Aucune transcription]", title="Transcription", style="cyan")
                group = Group(table, panel)
                live.update(group)
                time.sleep(0.1)
    except KeyboardInterrupt:
        print("\nArrêt en cours...")
        stop_listening(wait_for_stop=True)
        print("Programme arrêté.")
        stop = True  # ça stoppe la boucle proprement

# Lancer la fonction main()
main()


Calibrage du bruit ambiant...
Démarrage reconnaissance vocale et détection sentiment...


Output()


Arrêt en cours...
Programme arrêté.


In [8]:
!python --version

Python 3.11.5
